In [ ]:
import matplotlib.pyplot as plt
import sys
import numpy as np
from ase.io import read, write
from ase import Atoms
from ase.neighborlist import NeighborList
from scipy.spatial.distance import cdist
from tqdm import tqdm
from matplotlib import cm

In [ ]:
from matplotlib.colors import LogNorm
from mpl_toolkits.axes_grid1 import make_axes_locatable

In [ ]:
system_list = [
'anatase-100-nd-0',
'anatase-101-nd-0',
'anatase-110-nd-0',
'rutile-001-nd-0',
'rutile-011-nd-0',
'rutile-100-nd-0',#'rutile-101-nd-0',
'rutile-110-nd-0']

In [ ]:
surface_name_dict={"anatase-100":"anatase (100)",
                    "anatase-101":"anatase (101)",
                    "anatase-110": "anatase (110)",
                    "rutile-001": "rutile (001)",
                     "rutile-011": "rutile (011)", 
                    "rutile-100": "rutile (100)",
                    "rutile-110": "rutile (110)"}

In [ ]:
system_now = 'rutile-011-nd-0'

# Possible reshaping issue with 'rutile-100-nd-0', seems to not be 12 long but 11

nhydrogen = 128*2

In [ ]:
system_ref = 'anatase-101-nd-0' # always use this
with open('ref-anatase-101-nd-0-h-dis.npy', 'rb') as f:
    h_dis_all_ref = np.load(f)
    print(h_dis_all_ref.shape)

In [ ]:
colvar = np.genfromtxt(system_now+'/COLVAR')[1000:10000:10] # consistent with the gen of features
tt = 330 #K
kbt = 0.008314*tt # in kj/mol

print(len(colvar))
sys_weights = np.exp((colvar[:,2]+colvar[:,3])/kbt)

In [ ]:
h_weights = np.ones((len(sys_weights),nhydrogen))
for i in range(len(sys_weights)):
    h_weights[i,:] *= sys_weights[i]

In [ ]:
with open(system_now+'-h-dis-env.npy', 'rb') as f:
    h_dis_all = np.load(f)
    h_env_all = np.load(f)
    print(h_dis_all.shape)
    print(h_env_all.shape)

In [ ]:
%matplotlib inline 

In [ ]:
from asaplib.reducedim import Dimension_Reducers
from asaplib.plot import Plotters as asapPlotters

In [ ]:
reduce_dict = {}
"""
reduce_dict['pca'] = {
    "type": 'PCA',
    'parameter':{
        "n_components": 4}
}
"""
reduce_dict['kpca'] = {
    "type": 'SPARSE_KPCA',
    'parameter':{
        "n_components": 2,
        "n_sparse": 200, # no sparsification
        "kernel": {"first_kernel": {"type": 'cosine'}}
    }
}

dreducer = Dimension_Reducers(reduce_dict)

hcoord_ref = np.reshape(h_dis_all_ref,(-1,11))
hcoord_now = np.reshape(h_dis_all,(-1,12))

print(hcoord_ref.shape)
print(hcoord_now.shape)

# proj = dreducer.fit_transform(hcoord_now[:,[1,2,3,5,6,8,9,10]])

In [ ]:
dreducer.fit(hcoord_ref[:,[1,2,3,5,6,8,9,10]])
proj = dreducer.transform(hcoord_now[:,[1,2,3,5,6,8,9,10]])

print(proj.shape)

In [ ]:
# reshape
print(proj.shape[0]/nhydrogen)
h_proj = np.reshape( proj[:,[0,1]],(-1,nhydrogen,2))
print(np.shape(h_proj))

stride = 1
h_proj_sparse = h_proj[::stride,:,:]
np.shape(h_proj_sparse)

h_proj_sparse_all = np.reshape(h_proj_sparse[1:,:,:],(-1,2))

In [ ]:
# classify

cls_labels= ['H$_2$O$^{(> 1)}$','H-O$_t$','HO-Ti','H$_2$O-Ti','H$_2$O$^{(1)}$']

cls = np.zeros(len(hcoord_now))
for i,hh in enumerate(hcoord_now):
    if hh[3] >=5: 
        cls[i] = 0 # in the bulk
    elif hh[2] > 2: # OH
        if  hh[3] < 1.5:
            cls[i] = 1 # H on O(TiO2)
        else:
            cls[i] = 2 # OH on Ti
    elif hh[5]<3:
        cls[i] = 3 # H2O on Ti
    else:
        cls[i] = 4 # H2O close to slab but not on Ti
        
cls =  np.reshape( cls,(-1,nhydrogen))
np.shape(cls)

In [ ]:
    # the distance vector array: 
    # 0:central_atom, 1:rHTi, 2:rHH, 3:rHO_TiO2, 4:rOTi, 5:rOwTi, 6:r_O_1_z, 7:rHH_z, 8:v, 9:mu, 10:rOO, 11:rHH2
fig, axs = plt.subplots(2, 6, figsize=(16,9))

ax = axs[0,0]
plotcolor = hcoord_now[::1,1]
sc = ax.scatter(proj[::1,0], proj[::1,1], c=plotcolor, cmap='gnuplot', s=2, vmin=None, vmax=6)
cbar = plt.colorbar(sc, ax=ax)
cbar.set_label('H-Ti distance', rotation=90)
ax.get_xaxis().set_ticks([])
ax.get_yaxis().set_ticks([])

ax = axs[0,1]
plotcolor = hcoord_now[::1,2]
sc = ax.scatter(proj[::1,0], proj[::1,1], c=plotcolor, cmap='gnuplot', s=2, vmin=None, vmax=2)
cbar = plt.colorbar(sc, ax=ax)
cbar.set_label('H-H distance', rotation=90)
ax.get_xaxis().set_ticks([])
ax.get_yaxis().set_ticks([])
ax.text(.2, .99, system_now, ha='left', va='top', transform=ax.transAxes)


ax = axs[1,0]
plotcolor = hcoord_now[::1,3]
sc = ax.scatter(proj[::1,0], proj[::1,1], c=plotcolor, cmap='gnuplot', s=2, vmin=None, vmax=5)
cbar = plt.colorbar(sc, ax=ax)
cbar.set_label('H to O(TiO2) distance', rotation=90)
ax.get_xaxis().set_ticks([])
ax.get_yaxis().set_ticks([])

ax = axs[1,1]
plotcolor = hcoord_now[::1,4]
sc = ax.scatter(proj[::1,0], proj[::1,1], c=plotcolor, cmap='gnuplot', s=2, vmin=None, vmax=5)
cbar = plt.colorbar(sc, ax=ax)
cbar.set_label('nearest O-Ti distance', rotation=90)
ax.get_xaxis().set_ticks([])
ax.get_yaxis().set_ticks([])

ax = axs[0,2]
plotcolor = hcoord_now[::1,5]
sc = ax.scatter(proj[::1,0], proj[::1,1], c=plotcolor, cmap='gnuplot', s=2, vmin=None, vmax=None)
cbar = plt.colorbar(sc, ax=ax)
cbar.set_label('nearest O_w-Ti distance', rotation=90)
ax.get_xaxis().set_ticks([])
ax.get_yaxis().set_ticks([])

ax = axs[1,2]
plotcolor = hcoord_now[::1,6]
sc = ax.scatter(proj[::1,0], proj[::1,1], c=plotcolor, cmap='gnuplot', s=2, vmin=None, vmax=None)
cbar = plt.colorbar(sc, ax=ax)
cbar.set_label('OH_z', rotation=90)
ax.get_xaxis().set_ticks([])
ax.get_yaxis().set_ticks([])

ax = axs[0,3]
plotcolor = hcoord_now[::1,7]
sc = ax.scatter(proj[::1,0], proj[::1,1], c=plotcolor, cmap='gnuplot', s=2, vmin=None, vmax=None)
cbar = plt.colorbar(sc, ax=ax)
cbar.set_label('rHH-z', rotation=90)
ax.get_xaxis().set_ticks([])
ax.get_yaxis().set_ticks([])

ax = axs[1,3]
plotcolor = hcoord_now[::1,11]
sc = ax.scatter(proj[::1,0], proj[::1,1], c=plotcolor, cmap='gnuplot', s=2, vmin=None, vmax=2.5)
cbar = plt.colorbar(sc, ax=ax)
cbar.set_label('H-H(2) distance', rotation=90)
ax.get_xaxis().set_ticks([])
ax.get_yaxis().set_ticks([])


ax = axs[0,4]
plotcolor = hcoord_now[::1,8]
sc = ax.scatter(proj[::1,0], proj[::1,1], c=plotcolor, cmap='gnuplot', s=2, vmin=None, vmax=None)
cbar = plt.colorbar(sc, ax=ax)
cbar.set_label('v', rotation=90)
ax.get_xaxis().set_ticks([])
ax.get_yaxis().set_ticks([])

ax = axs[1,4]
plotcolor = hcoord_now[::1,9]
sc = ax.scatter(proj[::1,0], proj[::1,1], c=plotcolor, cmap='gnuplot', s=2, vmin=None, vmax=None)
cbar = plt.colorbar(sc, ax=ax)
cbar.set_label('u', rotation=90)
ax.get_xaxis().set_ticks([])
ax.get_yaxis().set_ticks([])

ax = axs[0, 5]
plotcolor = hcoord_now[::1,10]
sc = ax.scatter(proj[::1,0], proj[::1,1], c=plotcolor, cmap='gnuplot', s=2, vmin=None, vmax=3.5)
cbar = plt.colorbar(sc, ax=ax)
cbar.set_label('rOO', rotation=90)
ax.get_xaxis().set_ticks([])
ax.get_yaxis().set_ticks([])


rx = ax.get_xlim()
ry = ax.get_ylim()

fig.tight_layout()
plt.show()

#fig.savefig(system_now+'-H-analysis.png',dpi=100,rasterized=True)

In [ ]:
# build FES

    
H, xedges, yedges = np.histogram2d(proj[:,0], proj[:,1], 
                                   range=[[rx[0], rx[1]], [ry[0], ry[1]]],
                                   weights = h_weights.flatten(),
                                   normed = True,
                                   bins=80)

In [ ]:
FES = -kbt*np.log(H.T)
FES -= np.amin(FES)

In [ ]:
fig = plt.figure(constrained_layout=True,figsize=(12,6))
ax_dict = fig.subplot_mosaic(
    [
        ["a", "a","a", "b","c","d"],
        ["a", "a", "a","e","f","g"],
        ["a", "a", "a","h","i","j"],
    ],
)

ax=ax_dict["a"]
im = ax.contourf((xedges[1:]+xedges[:-1])/2., 
              (yedges[1:]+yedges[:-1])/2. ,
             FES, levels=np.arange(0,40),
             vmin=0, vmax=40, cmap='coolwarm')
#cbaxes = ax.inset_axes([0.2, 1.05, 0.75, 0.02])
#cbar= plt.colorbar(im, cax=cbaxes, orientation='horizontal')
cbar = plt.colorbar(im, ax=ax,aspect=50)
cbar.set_label('FES [kJ/mol]', rotation=90)

"""
for j in range(nhydrogen):# loop through points
    
    if np.amax(h_dis_all[:,j,2])>2.0: # only plot the transition path

        for i in range(1,len(h_proj_sparse)): # loop through the frames
            
            if cls[i,j] != cls[i-1,j] and cls[i,j]*cls[i-1,j]>0: 
            #if h_dis_all[i,j,5]<5 and h_dis_all[i-1,j,5]<5: #cls[i,j]*cls[i-1,j]>1: 
                point1 = h_proj_sparse[i-1,j,:]
                point2 = h_proj_sparse[i,j,:]
                x_values = [point1[0], point2[0]]
                y_values = [point1[1], point2[1]]
                ax.plot(x_values, y_values, 'gray', linestyle="-", alpha=0.6, lw=0.3)
"""

#ax=ax_dict["b"]
#ax.spines['top'].set_visible(False)
#ax.spines['right'].set_visible(False)
#ax.spines['bottom'].set_visible(False)
#ax.spines['left'].set_visible(False)
#ax.get_xaxis().set_ticks([])
#ax.get_yaxis().set_ticks([])


ax = ax_dict["b"]
plotcolor = cls
cmap = cm.get_cmap('jet', 5)
sc = ax.scatter(proj[::1,0], proj[::1,1], c=plotcolor, cmap=cmap, s=2, vmin=-0.5, vmax=4.5)
#cbaxes = ax.inset_axes([0.0, 1.1, 1.0, 0.02])
#cbar= plt.colorbar(sc, cax=cbaxes, ticks=[0, 1, 2, 3, 4], orientation='horizontal')
cbar = plt.colorbar(sc, ax=ax, ticks=[0, 1, 2, 3, 4])
cbar.ax.set_yticklabels(cls_labels)
#cbar.set_label('Classification',labelpad=-3) #, rotation=90)
cbar.set_label('Classification', rotation=90)




ax = ax_dict["c"]
plotcolor = hcoord_now[::1,2]
sc = ax.scatter(proj[::1,0], proj[::1,1], c=plotcolor, cmap='gnuplot', s=2, vmin=None, vmax=2)
cbar = plt.colorbar(sc, ax=ax)
cbar.set_label('H-H [$\mathrm{\AA}$]', rotation=90)

ax = ax_dict["d"]
plotcolor = hcoord_now[::1,1]
sc = ax.scatter(proj[::1,0], proj[::1,1], c=plotcolor, cmap='gnuplot', s=2, vmin=None, vmax=7)
cbar = plt.colorbar(sc, ax=ax)
cbar.set_label('H-Ti [$\mathrm{\AA}$]', rotation=90)

ax = ax_dict["e"]
plotcolor = hcoord_now[::1,3]
sc = ax.scatter(proj[::1,0], proj[::1,1], c=plotcolor, cmap='gnuplot', s=2, vmin=None, vmax=5)
cbar = plt.colorbar(sc, ax=ax)
cbar.set_label('H-O$_t$ [$\mathrm{\AA}$]', rotation=90)

ax = ax_dict["f"]
plotcolor = hcoord_now[::1,5]
sc = ax.scatter(proj[::1,0], proj[::1,1], c=plotcolor, cmap='gnuplot', s=2, vmin=None, vmax=7)
cbar = plt.colorbar(sc, ax=ax)
cbar.set_label('O$_\mathrm{w}$-Ti [$\mathrm{\AA}$]', rotation=90)

ax = ax_dict["g"]
plotcolor = hcoord_now[::1,6]
sc = ax.scatter(proj[::1,0], proj[::1,1], c=plotcolor, cmap='gnuplot', s=2, vmin=None, vmax=None)
cbar = plt.colorbar(sc, ax=ax)
cbar.set_label('OH$_z$ [$\mathrm{\AA}$]', rotation=90)

ax = ax_dict["h"]
plotcolor = hcoord_now[::1,8]
sc = ax.scatter(proj[::1,0], proj[::1,1], c=plotcolor, cmap='gnuplot', s=2, vmin=None, vmax=None)
cbar = plt.colorbar(sc, ax=ax)
cbar.set_label('Proton transfer $v$ [$\mathrm{\AA}$]', rotation=90)


ax = ax_dict["i"]
plotcolor = hcoord_now[::1,11]
sc = ax.scatter(proj[::1,0], proj[::1,1], c=plotcolor, cmap='gnuplot', s=2, vmin=None, vmax=2.5)
cbar = plt.colorbar(sc, ax=ax)
cbar.set_label('H-H$^{(2)}$ [$\mathrm{\AA}$]', rotation=90)

ax = ax_dict["j"]
plotcolor = hcoord_now[::1,10]
sc = ax.scatter(proj[::1,0], proj[::1,1], c=plotcolor, cmap='gnuplot', s=2, vmin=None, vmax=3.5)
cbar = plt.colorbar(sc, ax=ax,ticks=[2, 2.5, 3, 3.5])
cbar.set_label('rOO [$\mathrm{\AA}$]', rotation=90)
ax.get_xaxis().set_ticks([])
ax.get_yaxis().set_ticks([])

#fig.tight_layout()

#identify_axes(ax_dict)

for label,ax in ax_dict.items():
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)
    ax.spines['bottom'].set_visible(False)
    ax.spines['left'].set_visible(False)
    ax.get_xaxis().set_ticks([])
    ax.get_yaxis().set_ticks([])
    
import matplotlib.transforms as mtransforms

for label, ax in ax_dict.items():
    # label physical distance in and down:
    trans = mtransforms.ScaledTranslation(10/72, -5/72, fig.dpi_scale_trans)
    ax.text(0.0, 1.0, label, transform=ax.transAxes + trans,
            fontsize='large', verticalalignment='top')

fig.savefig(system_now+'-H-analysis.png',dpi=100,rasterized=True)
plt.show()

In [ ]:
np.shape(h_weights)

In [ ]:
# weighted transition rates (assuming V is quasi-static)

cl_transition = np.zeros((5,5))
print(cls.shape)
# I want to plot the evolution of H

count = 0
for j in range(nhydrogen):# loop through points    
    for i in range(1,np.shape(cls)[0]): # loop through the frames
        [c1, c2] = [int(cls[i,j]), int(cls[i-1,j])]
        cl_transition[c1,c2] += 1 # h_weights[i,j]/h_weights[i-1,j]
        if False: # c1 == 1 and c2 == 4:
            count += 1
            print(f"Timestep: {1000+(10*i):n}")
            print(f"Hydro Index: {h_dis_all[i, j, 0]:n}")
            if count == 50:
                raise ValueError
            
print(count)
print(cls.shape)
print(cl_transition)

for k in range(5):
    cl_norm = np.sum(cl_transition[k,:])
    cl_transition[k,:]/=cl_norm

In [ ]:
np.savetxt(system_now+'-K.dat',cl_transition)

In [ ]:
fig, ax = plt.subplots(1, figsize=(4,5))


for j in range(nhydrogen)[::3]:# loop through points
    
    
    if True: #np.amax(h_dis_all[:,j,2])>2.0 or np.amax(h_dis_all[:,j,8])>-0.5: # only plot the transition path

        for i in range(1,len(h_proj_sparse)): # loop through the frames
            
            if cls[i,j]*cls[i-1,j]>0: 
            #if h_dis_all[i,j,5]<5 and h_dis_all[i-1,j,5]<5: #cls[i,j]*cls[i-1,j]>1: 
                if cls[i,j]!=cls[i-1,j] or np.amax(h_dis_all[[i-1,i],j,8])>-0.5:
                    point1 = h_proj_sparse[i-1,j,:]
                    point2 = h_proj_sparse[i,j,:]
                    x_values = [point1[0], point2[0]]
                    y_values = [point1[1], point2[1]]
                    ax.plot(x_values, y_values, 'gray', linestyle="-", alpha=0.6, lw=0.2)

plotcolor = hcoord_now[::1,8]
sc = ax.scatter(proj[::1,0], proj[::1,1], c=plotcolor, cmap='gnuplot', s=2, vmin=-2, vmax=0)
#cbar = plt.colorbar(sc)
#cbar.set_label('Proton transfer coordinate', rotation=90)
cbaxes = ax.inset_axes([0.4, 0.1, 0.6, 0.02])
cbar= plt.colorbar(sc, cax=cbaxes, ticks=[-2,-1.5,-1,-0.5,0], orientation='horizontal')
cbar.set_label('Proton transfer $v$ [$\mathrm{\AA}$]',labelpad=-4)

"""
proj_important = np.asarray([ p for i,p in enumerate(proj) if hcoord_now[i,2] > 2 ])
ax.plot(proj_important[:,0], proj_important[:,1], 'o',
        markersize=2,markeredgewidth=0.1, markeredgecolor='k', markerfacecolor='none',ls='')
"""

"""
ax.text(0.2,-0.1,'bulk water', c='green',fontsize=14,weight="bold")
ax.text(0.15,0.15,'bulk water', c='green',fontsize=14,weight="bold")
#ax.text(0.22,-0.15,'bulk water', c='green',fontsize=14,weight="bold")

ax.text(-0.42,0.16,'H-Ot', c='green',fontsize=14,weight="bold")
ax.text(-0.1,0.0,'H2O...Ot', c='green',fontsize=14,weight="bold")
ax.text(-0.3,0.1,'H2O...Ot (H -> Ot)', c='green',fontsize=14,weight="bold")


ax.text(-0.3,-0.27,'H-O...Ti', c='green',fontsize=14,weight="bold")
ax.text(-0.24,-0.14,'H2-O...Ti', c='green',fontsize=14,weight="bold")
"""
ax.get_xaxis().set_ticks([])
ax.get_yaxis().set_ticks([])

ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
ax.spines['bottom'].set_visible(False)
ax.spines['left'].set_visible(False)

ax.text(.3, .99, surface_name_dict[system_now[:-5]], ha='left', va='top', transform=ax.transAxes)
fig.savefig(system_now+'-H-evolution.png',dpi=100,rasterized=True)
plt.show()

In [ ]:
print(np.shape(h_proj))
print(np.shape(h_env_all))

In [ ]:
test_frame = 1
for ii,a in enumerate(h_proj[test_frame, :, :]):
    if -0.3<a[0]<-0.1 and -0.2<a[1]<-0.15:
        print(ii)

In [ ]:
atom_dict = {
    1: ['b', 100],
    8: ['r', 200],
    22: ['gray', 400]
}


fig = plt.figure(figsize=(12, 8))
ax = fig.add_subplot(projection='3d')

probe_atom = 207

at_pos = h_env_all[test_frame,probe_atom,:,:]
ax.scatter(0,0,0, 
            s=100,
            c='cyan',
                alpha=1,
                edgecolors="k", # vmin=0, vmax=1
            )

for atom_now in at_pos:
    if atom_now[0] != 0:
        ax.scatter(
                atom_now[1], atom_now[2], atom_now[3], 
                s=atom_dict[atom_now[0]][1],
                c=atom_dict[atom_now[0]][0],
                alpha=1,
                edgecolors="k", # vmin=0, vmax=1
            )
    if atom_now[0] == 22:
        # plot the plane
        xx, yy = np.meshgrid([-2,2], [-2,2])
        z = xx*0+atom_now[3]
        ax.plot_surface(xx, yy, z, alpha=0.5,color='gray')
        
for i in range(len(at_pos)):
    for j in range(i):
        if np.linalg.norm(at_pos[i,1:4]-at_pos[j,1:4])< 1.3:
            #print(np.linalg.norm(at_pos[i,1:4]-at_pos[j,1:4]))
            ax.plot(np.array([at_pos[i,1],at_pos[j,1]]), 
                   np.array([at_pos[i,2],at_pos[j,2]]), 
                   np.array( [at_pos[i,3],at_pos[j,3]]),
                       ls='-',c='k')
        elif np.linalg.norm(at_pos[i,1:4]-at_pos[j,1:4])< 2.5 and (at_pos[j,0]==22 or at_pos[i,0]==22):
            #print(np.linalg.norm(at_pos[i,1:4]-at_pos[j,1:4]))
            ax.plot(np.array([at_pos[i,1],at_pos[j,1]]), 
                   np.array([at_pos[i,2],at_pos[j,2]]), 
                   np.array( [at_pos[i,3],at_pos[j,3]]),
                       ls='dotted',c='k')
        elif np.linalg.norm(at_pos[i,1:4]-at_pos[j,1:4])< 2.5 and (at_pos[j,0]*at_pos[i,0]==8):
            #print(np.linalg.norm(at_pos[i,1:4]-at_pos[j,1:4]))
            ax.plot(np.array([at_pos[i,1],at_pos[j,1]]), 
                   np.array([at_pos[i,2],at_pos[j,2]]), 
                   np.array( [at_pos[i,3],at_pos[j,3]]),
                       ls='dotted',c='k')


ax.set_xlim([-2,2])
ax.set_ylim([-2,2])
ax.set_zlim([-2,2])

# Hide grid lines
ax.grid(False)

# Hide axes ticks
#ax.set_xticks([])
#ax.set_yticks([])
#ax.set_zticks([])

# make the panes transparent
ax.xaxis.set_pane_color((1.0, 1.0, 1.0, 0.0))
ax.yaxis.set_pane_color((1.0, 1.0, 1.0, 0.0))
ax.zaxis.set_pane_color((1.0, 1.0, 1.0, 0.0))

#ax.set_axis_off()

plt.show()